In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
from datetime import datetime

### Read in data, view dtypes, etc.

In [3]:
# Create path for train.csv and weather.csv

path_train = '/Users/michaelshea/desktop/class/WestNile/WestNilePrediction/Assets/train.csv'
path_weather = '/Users/michaelshea/desktop/class/WestNile/WestNilePrediction/Assets/weather.csv'

In [4]:
# Read in dataframes

train = pd.read_csv(path_train)
weather = pd.read_csv(path_weather)

In [5]:
# Dtypes and columns of train_df 

print train.dtypes

Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
dtype: object


In [6]:
# Convert train.Date to datetime

train['Date'] = pd.to_datetime(train.Date)

In [7]:
# Dtypes and columns of weather_df

print weather.dtypes

Station          int64
Date            object
Tmax             int64
Tmin             int64
Tavg            object
Depart          object
DewPoint         int64
WetBulb         object
Heat            object
Cool            object
Sunrise         object
Sunset          object
CodeSum         object
Depth           object
Water1          object
SnowFall        object
PrecipTotal     object
StnPressure     object
SeaLevel        object
ResultSpeed    float64
ResultDir        int64
AvgSpeed        object
dtype: object


In [8]:
# Convert weather Station to object

weather['Station'] = weather.Station.apply(str)

In [9]:
# Convert weather Date column to datetime

weather['Date'] = pd.to_datetime(weather.Date)

In [10]:
# Set weather index to Date

weather.set_index('Date', inplace=True)

In [11]:
# Find Date range of weather data

print weather.index.min()
print weather.index.max()

2007-05-01 00:00:00
2014-10-31 00:00:00


In [12]:
# Was weather measured every day? Every weekday?
# According to internet, there were 2741 days between the dates above, 1886 excluding weekends and public holidays

# This proves weather not measured everyday (just something to note going forward):
print len(weather.index.unique())

1472


In [13]:
# Since test set includes 2008, 2010, 2012, and 2014, pull out these years from weather

weather = weather[(weather.index.year == 2007) | (weather.index.year == 2009) | (weather.index.year == 2011) | (weather.index.year == 2013)]

In [14]:
# Demonstrate above code worked:

print np.unique(weather.index.year)

[2007 2009 2011 2013]


### Calculate distance between weather station and mosquito trap

Since there are two weather stations, it might make sense to calculate the distance between each trap and the nearest weather station, and then use that station's weather data when we merge. The following link is what I found when I googled "calculate distance between two points latitude longitude python": http://www.johndcook.com/blog/python_longitude_latitude/

It uses this code:

In [15]:
import math
 
def distance_on_unit_sphere(lat1, long1, lat2, long2):

    # Convert latitude and longitude to spherical coordinates in radians
    degrees_to_radians = math.pi/180.0
 
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
 
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
 
    # Compute spherical distance from spherical coordinates.
 
    # For two locations in spherical coordinates
    # (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) =
    # sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length
 
    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) +
    math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )

    # Remember to multiply arc by the radius of the earth
    # in your favorite set of units to get length.
    return arc

In [16]:
# We could add a new column called closest_station to the train dataframe

station_1_lat = 41.995
station_1_lon = -87.933
station_2_lat = 41.786
station_2_lon = -87.752
dist_to_station_1 = []
dist_to_station_2 = []
closest_station = []

for i in range(len(train.index)):
    one = distance_on_unit_sphere(train.ix[i, 'Latitude'], train.ix[i, 'Longitude'], station_1_lat, station_1_lon)
    two = distance_on_unit_sphere(train.ix[i, 'Latitude'], train.ix[i, 'Longitude'], station_2_lat, station_2_lon)
    if dist_to_station_1 > dist_to_station_2:
        closest_station.append('1')
    else:
        closest_station.append('2')

In [17]:
# Now add closest_station column to weather stations to train dataframe

train['closest_station'] = closest_station

### Create unique identifier for table merging

In [18]:
# Add a unique identifier column to both train and weather dataframes by combining timestamp and weather station

train['date_station_id'] = train["Date"].map(str) + train["closest_station"]

weather['date_station_id'] = weather.index.map(str) + weather.Station.map(str)

In [19]:
# Check whether unique id works

train_dates = train.date_station_id.unique()

weather_dates = set(weather.date_station_id)

for day in train_dates:
    if day in weather_dates:
        print day

2007-05-29 00:00:002
2007-06-05 00:00:002
2007-06-26 00:00:002
2007-06-29 00:00:002
2007-07-02 00:00:002
2007-07-11 00:00:002
2007-07-18 00:00:002
2007-07-19 00:00:002
2007-07-25 00:00:002
2007-07-27 00:00:002
2007-08-01 00:00:002
2007-08-02 00:00:002
2007-08-03 00:00:002
2007-08-07 00:00:002
2007-08-08 00:00:002
2007-08-09 00:00:002
2007-08-15 00:00:002
2007-08-16 00:00:002
2007-08-17 00:00:002
2007-08-21 00:00:002
2007-08-22 00:00:002
2007-08-24 00:00:002
2007-08-28 00:00:002
2007-09-04 00:00:002
2007-09-05 00:00:002
2007-09-06 00:00:002
2007-09-12 00:00:002
2007-09-18 00:00:002
2007-09-19 00:00:002
2007-09-24 00:00:002
2007-09-25 00:00:002
2007-10-04 00:00:002
2007-10-09 00:00:002
2009-05-28 00:00:002
2009-06-02 00:00:002
2009-06-03 00:00:002
2009-06-05 00:00:002
2009-06-12 00:00:002
2009-06-15 00:00:002
2009-06-19 00:00:002
2009-06-22 00:00:002
2009-06-26 00:00:002
2009-06-29 00:00:002
2009-07-06 00:00:002
2009-07-10 00:00:002
2009-07-13 00:00:002
2009-07-17 00:00:002
2009-07-24 00

### Handle Missing Data

In [20]:
# 'M' in weather data is a missing value; convert all instances to np.nan
weather.replace(to_replace='M', value=np.nan, inplace=True)

In [21]:
# Sunrise and sunset values are omitted for the second reading of each day; this will allow us to backfill them all
# First convert empty cells to np.nan
weather.replace(to_replace='-', value=np.nan, inplace=True)

In [22]:
# Codesum column has many empty values; convert all empty cells to np.nan
weather.replace(to_replace=' ', value=np.nan, inplace=True)

In [23]:
# Now identify number of missing values in each column
weather.isnull().sum()

Station               0
Tmax                  0
Tmin                  0
Tavg                  6
Depart              736
DewPoint              0
WetBulb               4
Heat                  6
Cool                  6
Sunrise             736
Sunset              736
CodeSum             756
Depth               736
Water1             1472
SnowFall            736
PrecipTotal           2
StnPressure           4
SeaLevel              6
ResultSpeed           0
ResultDir             0
AvgSpeed              2
date_station_id       0
dtype: int64

In [24]:
# Drop columns with over half NANs, *except* Sunrise and Sunset (those will be backfilled)

weather.drop(['Depart', 'Depth', 'Water1', 'SnowFall', 'CodeSum'], axis=1, inplace=True)

In [25]:
# For remaining NANs, use backfill to impute values

weather.fillna(method='pad', limit=1, inplace=True)

In [26]:
# 'T' in weather data is trace precipitation; convert all instances to zero

weather.replace(to_replace='  T', value=0, inplace=True)

In [27]:
weather.dtypes

Station             object
Tmax                 int64
Tmin                 int64
Tavg                object
DewPoint             int64
WetBulb             object
Heat                object
Cool                object
Sunrise             object
Sunset              object
PrecipTotal         object
StnPressure         object
SeaLevel            object
ResultSpeed        float64
ResultDir            int64
AvgSpeed            object
date_station_id     object
dtype: object

In [28]:
# Now that missing values and strings have been removed from data, convert columns to numeric

weather['Tavg'] = pd.to_numeric(weather.Tavg)
weather['WetBulb'] = pd.to_numeric(weather.WetBulb)
weather['Heat'] = pd.to_numeric(weather.Heat)
weather['Cool'] = pd.to_numeric(weather.Cool)
weather['Sunrise'] = pd.to_numeric(weather.Sunrise)
weather['Sunset'] = pd.to_numeric(weather.Sunset)
weather['PrecipTotal'] = pd.to_numeric(weather.PrecipTotal)
weather['StnPressure'] = pd.to_numeric(weather.StnPressure)
weather['SeaLevel'] = pd.to_numeric(weather.SeaLevel)
weather['AvgSpeed'] = pd.to_numeric(weather.AvgSpeed)

In [30]:
weather.to_csv('../Assets/weather_clean.csv', encoding= 'utf-8')

### EDA